In [3]:
%%HTML
<script src="require.js"></script>

In [4]:
from IPython.display import HTML
HTML('''<script src="https://cdnjs.cloudflare.com/ajax/libs/jquery/2.0.3/jquery.min.js "></script><script>
code_show=true; 
function code_toggle() {
if (code_show){
$('div.jp-CodeCell > div.jp-Cell-inputWrapper').hide();
} else {
$('div.jp-CodeCell > div.jp-Cell-inputWrapper').show();
}
code_show = !code_show
} 
$( document ).ready(code_toggle);</script><form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>
''')

<div align="center">
<img src="Figures/Title.png" />
</div>

<div style="background-color: #270f47; color: #eed47c; padding: 15px; text-align: left; border-radius: 5px;">

# I. Table of Contents

</div>

<div style="background-color: #270f47; color: #eed47c; padding: 15px; text-align: left; border-radius: 5px;">

# II. Introduction

</div>

<div style="color: #270f47; padding: 15px; text-align: left; border-radius: 5px;">

## Problem Statement

</div>

<div style="color: #270f47; padding: 15px; text-align: left; border-radius: 5px;">

## Motivation

</div>

<div style="background-color: #270f47; color: #eed47c; padding: 15px; text-align: left; border-radius: 5px;">

# III. Methodology Overview

</div>

<div align="center">
<img src="Figures/Methodology Overview.png" />
</div>

The figure above shows the methodology overview of this study.
Our methodology consists of a four step process starting with the collection of historical Binance data, followed by exploratory data analysis, data processing, and the results and discussion. 
Under the results and discussion portion, we would also discuss the potential impacts this can have on investors, the perceived benefits and disadvantages, our recommendations and conclusions. 

Below would be a short summary of each of the different steps:
1. **Data Collection**: Here, we collect publicly available historical Binance data focusing on the closing price. Since the dataset is very big, we stored it in a Spark DataFrame.
2. **Exploratory Data Analysis**: Here, we prepared and cleaned the dataset to ensure that it is suitable for further analysis. EDA is used to understand the behavior of the closing price for different classifications: base currencies, quote currencies, and even their classifications. These would be further discussed later on.
3. **Data Processing**: In this step, we want to get answers for our research problem which is to find out the volatility, price stability, and trading activity patterns among the three classifications: cryptocurrency, fiat-based tokens, and stablecoin.
4. **Results and Discussion**: Based on what we find, we want to discuss what this could mean for investors, mention our perceived benefits and disadvantages, and recommend further steps. Afterwards, we will summarize findings in the recommendations and conclusion section.

<div style="background-color: #270f47; color: #eed47c; padding: 15px; text-align: left; border-radius: 5px;">

# IV. Data Collection

</div>

<div style="color: #270f47; padding: 15px; text-align: left; border-radius: 5px;">

## Data Source

</div>

We gathered data from the Binance Full History dataset available on the Jojie-collected public datasets of the Asian Institute of Management. 
The directory is as follows: `/mnt/data/public/binance-full-history`.
This dataset consists of more than 30 Parquet files that correspond to the information per base currency in its quote currency.
The dataset, as a whole, takes up 33 GB of data, so it would not be feasible to use the typical Python operations to handle all of it. 
Instead, we used Apache Spark, and created a Spark DataFrame just to contain all the information we need.

<div style="color: #270f47; padding: 15px; text-align: left; border-radius: 5px;">

## Data Description

</div>

The following features are consistent within all Parquet files in the Binance Full History dataset:

| Feature Name | Data Description | Data Type |
|-------------|-----------------|-----------|
| open | Opening price of the trading period | float |
| high | Highest price during the trading period | float |
| low | Lowest price during the trading period | float |
| close | Closing price of the trading period | float |
| volume | Total volume of base asset traded | float |
| quote_asset_volume | Total volume of quote asset traded | float |
| number_of_trades | Total number of trades executed | integer |
| taker_buy_base_asset_volume | Volume of base asset bought by takers | float |
| taker_buy_quote_asset_volume | Volume of quote asset bought by takers | float |
| open_time | Timestamp when the trading period started | timestamp_ntz |

<div style="color: #270f47; padding: 15px; text-align: left; border-radius: 5px;">

## Data Collection and DataFrame Creation

</div>

Since there are a lot of files, and the entire dataset sums to a total of 33 GB, we had to use Apache Spark to handle the data. 
To summarize the code below, we created a Spark DataFrame from all the information available.
Here, we created new features: `base_currency`, `quote_currency`, and `classification`. 
These three are not anywhere in the previously shown data dictionary because they are information you can only get from the file names of the Parquet files.

In [ ]:
# Directory containing the parquet files
directory = '/mnt/data/public/binance-full-history'
files = [f for f in os.listdir(directory) if f.endswith('.parquet')]

base_quote_counts = {}

for file in files:
    # Get base and quote per file name
    base_currency = file.split('-')[0]
    quote_currency = file.split('-')[1].replace('.parquet', '')
    
    if base_currency not in base_quote_counts:
        base_quote_counts[base_currency] = set()
    
    base_quote_counts[base_currency].add(quote_currency)

# Convert to counts
base_counts = {base: len(quotes) for base, quotes in base_quote_counts.items()}

# Create dataframe for plotting
counts_df = pd.DataFrame({
    'base_currency': list(base_counts.keys()),
    'quote_count': list(base_counts.values())
})
quote_count_distribution = counts_df['quote_count'].value_counts().sort_index()

# Plotting
plt.figure(figsize=(15, 8))
bars = plt.bar(
    quote_count_distribution.index, quote_count_distribution.values
)

for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + 0.1,
             f'{int(height)}', ha='center', va='bottom')

plt.xlabel('Number of Quote Currencies')
plt.ylabel('Number of Base Currencies')
plt.title('Distribution of Base Currencies by Number of Quote Currencies')
plt.grid(axis='y', alpha=0.3)
plt.xticks(quote_count_distribution.index)

# Saving the figure
plt.tight_layout()
plt.savefig(
    'Figures/Distribution of Base Currencies by Number of Quote Currencies.png'
)
plt.show()

# Get top base currencies
top_bases = counts_df.nlargest(75, 'quote_count')

As seen here, there are a lot of files, but most of them only have one quote currency.
There are only a few with more than 10 currencies.
Since we want to do a comparative analysis on the different quote currencies and their relationship to the base currency, we will only look at the top 75 base currencies so that we have files that we can compare with others.  

In [ ]:
# Getting the total size of the top base currencies
total_size_bytes = 0
total_size_gb = 0

for _, row in top_bases.iterrows():
    base_currency = row['base_currency']
    quote_count = row['quote_count']
    
    # Find all files for this base currency
    base_files = [f for f in files if f.startswith(f"{base_currency}-")]
    
    # Calculate total size for this base currency
    base_total_size = 0
    for file in base_files:
        file_path = os.path.join(directory, file)
        base_total_size += os.path.getsize(file_path)
    
    # Convert to human readable format
    base_size_mb = base_total_size / (1024 * 1024)
    base_size_gb = base_total_size / (1024 * 1024 * 1024)
    
    # Add to total
    total_size_bytes += base_total_size
    total_size_gb += base_size_gb
    
# Calculate overall totals
total_size_mb = total_size_bytes / (1024 * 1024)
total_all_files_bytes = sum(os.path.getsize(os.path.join(directory, f)) for f in files)
total_all_files_gb = total_all_files_bytes / (1024 * 1024 * 1024)

print(f"{'TOTAL (Top 10)':<24} {total_size_gb:>8.2f} GB ({total_size_mb:>.1f} MB)")
print(f"{'Percentage of All Data':<24} {(total_size_gb/total_all_files_gb)*100:>8.1f}%")
print(f"{'TOTAL (All Files)':<24} {total_all_files_gb:>8.2f} GB")

As shown in the code above, the top 75 currencies already consist of at least 16 GB which is the minimum file size requirement for this report.

In [ ]:
# Initialize Spark Session
spark = SparkSession.builder.appName("TopBasesAnalysis").getOrCreate()

# Directory containing the parquet files
directory = '/mnt/data/public/binance-full-history'
files = [f for f in os.listdir(directory) if f.endswith('.parquet')]

# Read the main Binance data
binance = spark.read.parquet(directory)

# Base Classifications (cryptocurrencies, stablecoins, and fiats)
cryptos = [
    '1INCH', 'AAVE', 'ACM', 'ADA', 'ADADOWN', 'ADAUP', 'ADX', 'AE', 'AERGO', 'AGI',
    'AION', 'AKRO', 'ALGO', 'ALICE', 'ALPHA', 'AMB', 'ANKR', 'ANT', 'APPC', 'AR',
    'ARDR', 'ARK', 'ARN', 'ARPA', 'ASR', 'AST', 'ATA', 'ATM', 'ATOM', 'AUCTION',
    'AUDIO', 'BCH', 'BEAM', 'BEL', 'BETA', 'BETH', 'BIFI', 'BLZ', 'BNB', 'BNBDOWN',
    'BNBUP', 'BNT', 'BQX', 'BRD', 'BTC', 'BTCDOWN', 'BTCST', 'BTCUP', 'BTG', 'BTS',
    'BTT', 'BURGER', 'BZRX', 'C98', 'CAKE', 'CTK', 'CTSI', 'CTXC', 'CVC', 'CVP',
    'DAR', 'DASH', 'DATA', 'DCR', 'DEGO', 'DENT', 'DEXE', 'DF', 'DGB', 'DGD', 'DIA',
    'DLT', 'DNT', 'DOCK', 'DODO', 'DOGE', 'DOT', 'DOTDOWN', 'DOTUP', 'DREP', 'DUSK',
    'DYDX', 'EDO', 'EGLD', 'ELF', 'ENG', 'ENJ', 'EOS', 'ETC', 'ETH', 'FET', 'FIL',
    'FIO', 'FIRO', 'FIS', 'FLM', 'FLOW', 'FOR', 'FORTH', 'FRONT', 'FTM', 'FTT',
    'FUEL', 'FUN', 'FXS', 'GALA', 'GAS', 'GHST', 'GLM', 'GNT', 'GO', 'GRS', 'GRT',
    'GTC', 'GTO', 'GVT', 'GXS', 'HARD', 'HBAR', 'HC', 'HIVE', 'HNT', 'HOT', 'ICP',
    'ICX', 'IOTA', 'KSM', 'LAZIO', 'LEND', 'LINA', 'LINK', 'LINKDOWN', 'LINKUP',
    'LIT', 'LOOM', 'LPT', 'LRC', 'LSK', 'LTC', 'LTCDOWN', 'LTCUP', 'LTO', 'LUN',
    'LUNA', 'MANA', 'MASK', 'MATIC', 'MBL', 'MBOX', 'MCO', 'MDA', 'MDT', 'MDX',
    'MFT', 'MINA', 'MIR', 'MITH', 'NEO', 'OCEAN', 'OG', 'OGN', 'OM', 'OMG', 'ONE',
    'ONG', 'ONT', 'ORN', 'OST', 'OXT', 'PAXG', 'PERL', 'PERP', 'PHA', 'PHB', 'PIVX',
    'PNT', 'POA', 'POE', 'POLS', 'POLY', 'POND', 'POWR', 'PPT', 'PROM', 'PROS',
    'PSG', 'PUNDIX', 'QKC', 'QLC', 'QNT', 'QSP', 'QTUM', 'RAMP', 'RCN', 'RDN',
    'REEF', 'REN', 'REP', 'REQ', 'SNGLS', 'SNM', 'SNT', 'SNX', 'SOL', 'SPARTA',
    'SRM', 'STEEM', 'STMX', 'STORJ', 'STORM', 'STPT', 'STRAT', 'STRAX', 'STX',
    'SUN', 'SUPER', 'SUSHI', 'SUSHIDOWN', 'SUSHIUP', 'SXP', 'SXPUP', 'SYS', 'TCT',
    'TFUEL', 'THETA', 'TKO', 'TLM', 'TNB', 'TNT', 'TOMO', 'TORN', 'TRB', 'TROY',
    'TRX', 'VET', 'VIA', 'VIB', 'VIBE', 'VIDT', 'VITE', 'VTHO', 'WABI', 'WAN',
    'WAVES', 'WBTC', 'WIN', 'WING', 'WNXM', 'WPR', 'WRX', 'WTC', 'XEC', 'XEM',
    'XLM', 'XMR', 'XRP', 'XRPDOWN', 'XRPUP', 'XTZ']
stablecoins = ['USDT','USDC','BUSD','TUSD','DAI','PAX','BIDR','IDRT']
fiats = ['EUR','GBP','AUD','TRY','BRL','RUB','NGN','UAH']

classified = {
    "cryptocurrency": [],
    "stablecoin": [],
    "fiat_backed": []
}

for f in files:
    pair = f.replace('.parquet','')
    base, quote = pair.split('-')
    
    if base in cryptos:
        classified["cryptocurrency"].append(f)
    elif base in stablecoins:
        classified["stablecoin"].append(f)
    elif base in fiats:
        classified["fiat_backed"].append(f)
    else:
        classified["cryptocurrency"].append(f)

classification = {}
for category, file_list in classified.items():
    for f in file_list:
        pair = f.replace(".parquet", "")
        classification[pair] = category

# Add stock column and create classification DataFrame
binance_with_stock = binance.withColumn(
    "stock",
    F.regexp_extract(F.input_file_name(), r'([^/]+)\.parquet', 1)
)

class_rows = [
    (pair, category)
    for pair, category in classification.items()
]

class_df = spark.createDataFrame(class_rows, ["stock", "classification"])

# Create the main DataFrame with classifications
df = (
    binance_with_stock
    .join(class_df, on="stock", how="left")
    .withColumn("base_currency", F.split(F.col("stock"), "-")[0])
    .withColumn("quote_currency", F.split(F.col("stock"), "-")[1])
)

# Final DataFrame with only top base currencies and closing price
top_base_list = top_bases['base_currency'].to_list()
df = (
    df
    .filter(F.col("base_currency").isin(top_base_list))
    .select(
        "stock",
        "base_currency",
        "quote_currency", 
        "open_time",
        "close",
        "classification"
    )
)

# Show the result
print("Final DataFrame Schema:")
df.printSchema()
df.cache()

# Save the filtered data just in case
df.write.parquet(
    "Files/top_bases_closing_prices.parquet"
)

The schema of the new Spark DataFrame is as follows:

| Feature | Description | Data Type |
|---------|-------------|-----------|
| **stock** | The complete trading pair identifier combining base and quote currencies (e.g., "BTC-USDT") | string |
| **base_currency** | The primary cryptocurrency or asset being traded (e.g., "BTC", "ETH", "BNB") | string |
| **quote_currency** | The currency used to price the base asset (e.g., "USDT", "BTC", "BUSD") | string |
| **open_time** | The timestamp indicating when the trading period began | timestamp_ntz |
| **close** | The final trading price of the asset at the end of the period | float |
| **classification** | Categorical classification of the base currency (cryptocurrency/stablecoin/fiat_backed) | string |

Note that the stock feature is only named as such due to a lack of a better name for it, but, in its essence, it is the same as the file name.

<div style="background-color: #270f47; color: #eed47c; padding: 15px; text-align: left; border-radius: 5px;">

# V. Exploratory Data Analysis

</div>

<div style="background-color: #270f47; color: #eed47c; padding: 15px; text-align: left; border-radius: 5px;">

# VI. Data Processing

</div>

In this portion, we are trying to fulfill our goals where we compare volatility, price stability, and trading activity patterns.

<div style="background-color: #270f47; color: #eed47c; padding: 15px; text-align: left; border-radius: 5px;">

# VII. Results and Discussion

</div>

<div style="background-color: #270f47; color: #eed47c; padding: 15px; text-align: left; border-radius: 5px;">

# VIII. Conclusion and Recommendations

</div>

<div style="color: #270f47; padding: 15px; text-align: left; border-radius: 5px;">

## Conclusion

</div>

<div style="color: #270f47; padding: 15px; text-align: left; border-radius: 5px;">

## Recommendations

</div>

### Limitations of the Study

insert your text here

### Recommendations for Readers

insert your text here

### Future Work

insert your text here

<div style="background-color: #270f47; color: #eed47c; padding: 15px; text-align: left; border-radius: 5px;">

# IX. References

</div>

<div style="background-color: #270f47; color: #eed47c; padding: 15px; text-align: left; border-radius: 5px;">

# X. Supplementary

</div>